In [4]:
import pymupdf
import pymupdf as fitz

# Ruta al archivo PDF
path = "C:\\Users\\ELI\\Downloads\\irbookonlinereading.pdf"
doc = pymupdf.open(path)
data=[]
for i, page in enumerate(doc):
    text = page.get_text().encode('utf-8')
    if text.strip():
        data.append({
            "page": i + 1,
            "raw": str(text).strip()
        })
# Convertir a DataFrame
import pandas as pd
df = pd.DataFrame(data)
df

,page,raw
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...
1,2,b'Online edition (c)\n2009 Cambridge UP\n'
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...
...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...


In [5]:
# Verificar si el PDF tiene tabla de contenidos (TOC)
toc = doc.get_toc()

if toc:
    print(f"¡El PDF tiene {len(toc)} elementos en la tabla de contenidos!")
    print("\nPrimeras 10 entradas:")
    for i, (level, title, page) in enumerate(toc[:100]):
        indent = "  " * (level - 1)
        print(f"{indent}Nivel {level}: {title} (Página {page})")
else:
    print("El PDF no tiene tabla de contenidos estructurada")

¡El PDF tiene 256 elementos en la tabla de contenidos!

Primeras 10 entradas:
Nivel 1: List of Tables (Página 15)
Nivel 1: List of Figures (Página 19)
Nivel 1: Table of Notation (Página 27)
Nivel 1: Preface (Página 31)
Nivel 1: Boolean retrieval (Página 38)
  Nivel 2: An example information retrieval problem (Página 40)
  Nivel 2: A first take at building an inverted index (Página 43)
  Nivel 2: Processing Boolean queries (Página 47)
  Nivel 2: The extended Boolean model versus ranked retrieval (Página 51)
  Nivel 2: References and further reading (Página 54)
Nivel 1: The term vocabulary and postings lists (Página 56)
  Nivel 2: Document delineation and character sequence decoding (Página 56)
    Nivel 3: Obtaining the character sequence in a document (Página 56)
    Nivel 3: Choosing a document unit (Página 57)
  Nivel 2: Determining the vocabulary of terms (Página 59)
    Nivel 3: Tokenization (Página 59)
    Nivel 3: Dropping common terms: stop words (Página 64)
    Nivel 3: Normali